<a href="https://colab.research.google.com/github/jcandane/CI_Theory/blob/main/CIround2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from itertools import combinations, permutations

π = np.pi
α = 0.007297352
c = 1.0/α
np.set_printoptions(precision=4, linewidth=200, threshold=2000, suppress=True)

### Manipulate MO -> SO etc...

In [ ]:
def Ext(A, N, numtype=np.int8):
    if A.ndim == 1:
        return np.einsum("I, j -> Ij", A, np.ones( N, dtype=numtype))
    if A.ndim == 2:
        return np.einsum("uI, j -> uIj", A, np.ones( N, dtype=numtype))

def SpinOuterProduct(A, B, stack=False):
  ΛA = np.einsum("Ii, J -> IJi", A, np.ones(B.shape[0], dtype=np.int8)).reshape( (A.shape[0]*B.shape[0], A.shape[1]) )
  ΛB = np.einsum("Ii, J -> JIi", B, np.ones(A.shape[0], dtype=np.int8)).reshape( (A.shape[0]*B.shape[0], B.shape[1]) )
  
  if stack:
    return np.array([ΛA,ΛB])
  else:
    return ΛA, ΛB

def combine_arrays(Λ_sample, V_sample):

    if V_sample.size == 0:
        return Λ_sample

    else:
        l = V_sample.shape[1] + Λ_sample.shape[1] # length of new array
        m = V_sample.shape[1] # level of excitation
        a_new = np.empty((V_sample.shape[0] * Λ_sample.shape[0], V_sample.shape[1] + Λ_sample.shape[1] ), dtype=np.int8)
        a_new[:,:l - m ] = np.repeat(Λ_sample, len(V_sample), axis=0)
        a_new[:, l - m:] = np.repeat(V_sample, len(Λ_sample), axis=1).reshape((len(Λ_sample)*len(V_sample), V_sample.shape[1]), order="F")

        return np.sort(a_new, axis=1)

def CIunique(dets):
    BB = np.unique( ( dets.swapaxes(0,1)).reshape(dets.shape[1], dets.shape[0]*dets.shape[2], order="C") , axis=0)
    return BB.reshape(BB.shape[0], 2, BB.shape[1]//2, order="C").swapaxes(0,1)

def TEST_electronnumber(AA): ## with B_sIp
    arr = np.einsum("sIp -> I", AA)
    if np.all(arr == arr[0]):
        return True
    else:
        return arr

def TEST_unquineness(AA): ## with B_sIp
    return AA.shape[1] == len( np.unique( (AA.swapaxes(0,1)).reshape((AA.shape[1], 2*AA.shape[2])) ,axis=1 ) )

### Combinatorics

In [ ]:
### for a single reference in binary, for 1 spin get all m-excitations, yield binary
def ΛMOgetB(Λ, N_mo):
  "Given Λ (i occupied orbitals for each determinant) get B (binary rep.)"

  Binary  = np.zeros((Λ.shape[0], N_mo), dtype=np.int8)
  for I in range(len(Binary)):
      Binary[I, Λ[I,:]] = 1

  return Binary



def MO_Excitation(MO_p, m, SF=False):
    """ 
    m-level CI excitations on an MO 
    m=0 should return the reference
    """

    occ = np.asarray([ np.where(MO_p == 1)[0] ]) ## occupied space
    vac = np.asarray([ np.where(MO_p == 0)[0] ]) ## vacant   space

    if SF: ### do SF-calculation
        if m < 0: ## if m is negative, then 
            Occ_choices = np.asarray( list(combinations(  occ[0], len(occ[0]) + m ) ) , dtype=np.int8)
            return ΛMOgetB(Occ_choices, len(MO_p)) 
        else:
            Vac_choices = np.asarray( list(combinations(  vac[0], m ) ) , dtype=np.int8)
            return ΛMOgetB(combine_arrays(occ, Vac_choices), len(MO_p)) 
    else: ### do CI-calculation
        #print(m, len(occ[0]) - m )
        Occ_choices = np.asarray( list(combinations(  occ[0], len(occ[0]) - m ) ) , dtype=np.int8)
        Vac_choices = np.asarray( list(combinations(  vac[0], m ) ) , dtype=np.int8)
        return ΛMOgetB(combine_arrays(Occ_choices, Vac_choices), len(MO_p))

def SO_Excitation(SO_ref, CIm=None, SFm=None):
    """ Given a reference, and CIm and SFm vectors get Excitation Slater Det. 
    r_sp = reference
    """

    refA = SO_ref[0]
    refB = SO_ref[1]
    D    = np.array([])
    SF   = np.array([])

    ### CIm
    if CIm is not None:
        first = True
        for m in CIm:
            for j in range(m+1): ## e.g. if m =3, then j in {0,1,2,3}
                ## count number of vaccant orbitals
                ## the if statement goes over occupied and vaccant orbitals!!!
                vacA = len(np.where(refA == 0)[0])
                vacB = len(np.where(refB == 0)[0])
                occA = len(np.where(refA == 1)[0])
                occB = len(np.where(refB == 1)[0])
                if vacA+1 > j and vacB+1 > m-j and occA > j and occB > m-j:
                    A = MO_Excitation(refA,   j, SF=False)
                    B = MO_Excitation(refB, m-j, SF=False)
                    C = SpinOuterProduct(A, B, stack=True)

                    if first:
                        D = C
                        first = False
                    else:
                        D = np.append(D, C, axis=1)
                else:
                    continue

    ### SFm
    if SFm is not None:
        first = True
        a  = np.sum(refA)
        b  = np.sum(refB)
        c  = np.abs(np.sum(refA - 1))
        d  = np.abs(np.sum(refB - 1))
        for m in SFm:
            if m < np.amin([a,b,c,d]) + 1:
                A = MO_Excitation(refA,  m, SF=True)
                B = MO_Excitation(refB, -m, SF=True)
                C = SpinOuterProduct(A,  B, stack=True)

                A = MO_Excitation(refA, -m, SF=True)
                B = MO_Excitation(refB,  m, SF=True)
                E = SpinOuterProduct(A,  B, stack=True)

                SF = np.concatenate((C, E), axis=1)

            else:
              continue

    if D.size == 0 and SF.size == 0: 
        return None

    if D.size == 0 and SF.size != 0:
        return SF

    if SF.size == 0 and D.size != 0:
        return D

    if SF.size != 0 and D.size != 0:
        return np.concatenate((D, SF), axis=1)

def ECI(MR, RAS=None, CIm=None, SFm=None):
    """ Compute MRRASSFCI """

    if RAS is None:
        RAS = np.arange(0, MR.shape[2], dtype=np.int8)

    first = True
    for reference in MR:
        RASref = reference[:, RAS]
        RASCI  = SO_Excitation(RASref, CIm=CIm, SFm=SFm)
        if RASCI is not None:
            loop   = Ext(reference, RASCI.shape[1]).swapaxes(1,2)
            if first:
                loop[:, :, RAS] = RASCI
                out = loop
                first = False

            else:
                out = np.append(out, loop, axis=1)
                out = CIunique(out)
                
        else: 
            continue

    if RASCI is None:
        return MR
    else:
        return out

In [ ]:
for j in range(3+1):
  vacA = 2
  vacB = 1
  if vacA+1 > j and vacB+1 > 3-j:
    print(j, 3-j)

2 1


### Tests

In [ ]:
ref1_sp = np.array([[1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 0, 0, 0]], dtype=np.int8)
ref2_sp = np.array([[0, 1, 0, 1, 1, 0, 0], [0, 1, 1, 0, 0, 1, 0]], dtype=np.int8)
ref3_sp = np.array([[1, 1, 1, 0, 1, 0, 0, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0, 0]], dtype=np.int8)
ref4_sp = np.array([[1, 1, 0, 0, 0], [1, 1, 0, 0, 0]], dtype=np.int8)
ref5_sp = np.array([[[1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 0, 0, 0]], [[1, 1, 0, 1, 1, 0, 1], [0, 1, 1, 1, 0, 1, 0]]], dtype=np.int8)
ref6_sp = np.array([[[1, 1, 0, 0, 0], [1, 1, 0, 0, 0]], [[1, 0, 0, 1, 0], [1, 1, 0, 0, 0]], [[1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]], dtype=np.int8)

In [ ]:
AA = ECI(ref5_sp, RAS=None, CIm=[3], SFm=None)
print(TEST_electronnumber(AA))
print(TEST_unquineness(AA))

AA = ECI(ref6_sp, RAS=None, CIm=[3,4], SFm=None)
print(TEST_electronnumber(AA))
print(AA)
##print(TEST_unquineness(AA))

AA = ECI(np.asarray([ref4_sp]), RAS=None, CIm=[3], SFm=None)
print(TEST_electronnumber(AA))

#print(AA.shape)
#print(TEST_unquineness(AA))

AA = ECI(np.asarray([ref1_sp]), RAS=None, CIm=[3], SFm=[1])
print(TEST_electronnumber(AA))
print(TEST_unquineness(AA))

True
True
True
[[[1 1 0 0 0]
  [1 1 0 0 0]]

 [[1 0 0 1 0]
  [1 1 0 0 0]]

 [[1 1 0 0 0]
  [1 0 0 0 1]]]
True
True
True


## MO Excitation

In [ ]:
def MO_X(MO_p, m, SF=False):
    """ 
    m-level CI excitations on an MO 
    m=0 should return the reference
    """

    occ = np.asarray([ np.where(MO_p == 1)[0] ]) ## occupied space
    vac = np.asarray([ np.where(MO_p == 0)[0] ]) ## vacant   space

    if SF: ### do SF-calculation
        if m < 0 and abs(m) <= len(occ[0]): ## if m is negative, then we do all combinations deleting occupied orbital(s) 
            Occ_choices = np.asarray( list(combinations(  occ[0], len(occ[0]) + m ) ) , dtype=np.int8)
            return ΛMOgetB(Occ_choices, len(MO_p))
        if m == 0: ## if m = 0 , return reference
            return np.asarray([MO_p])
        if m > 0 and m <= len(vac[0]): ## if m is positive, we will do all combinations by adding occupied orbital(s) 
            Vac_choices = np.asarray( list(combinations(  vac[0], m ) ) , dtype=np.int8)
            return ΛMOgetB(combine_arrays(occ, Vac_choices), len(MO_p))
        else:
            return None

    else: ### MO-conserved-calculation
        ## excitation order must be less than avaiable orbitals
        if m <= len(occ[0]) and m <= len(vac[0]): 
            Occ_choices = np.asarray( list(combinations(  occ[0], len(occ[0]) - m ) ) , dtype=np.int8)
            Vac_choices = np.asarray( list(combinations(  vac[0], m ) ) , dtype=np.int8)
            return ΛMOgetB(combine_arrays(Occ_choices, Vac_choices), len(MO_p))
        else:
            return None

reff = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0, 0]) ## ref3_sp[0] #
print(reff)
MOexcite = MO_X(reff, 0, SF=True)

### Test 1
print( np.all(np.einsum("Ip -> I", MOexcite) == np.sum(reff)) ) ## 5 electrons

### Test 2
print( MOexcite.shape[0] == (np.unique( MOexcite, axis=1)).shape[0] )


print( MO_X(reff, -6, SF=True) )

[1 1 1 0 1 0 0 1 0 0]
True
True
None


# SO Excitation

In [ ]:
def SO_X_m(ref_SO, m):

    D     = None
    first = True
    for j in range(m+1):
        A = MO_X(ref_SO[0],   j)
        B = MO_X(ref_SO[1], m-j)
        if A is not None and B is not None:
            C = SpinOuterProduct(A, B, stack=True)
            if first:
                D = C
                first = False
            else:
                D = np.append(D, C, axis=1)
        else:
            continue
    return D

SO_X_m(ref3_sp, 1).shape

(2, 40, 9)

In [ ]:
def SO_X_SF(ref_SO, m):
    C = None
    D = None
    ## +3 & -3
    A = MO_X(ref_SO[0],  m, SF=True)
    B = MO_X(ref_SO[1], -m, SF=True)
    if A is not None and B is not None:
        C = SpinOuterProduct(A, B, stack=True)
    
    ## -3 & +3
    A = MO_X(ref_SO[0], -m, SF=True)
    B = MO_X(ref_SO[1],  m, SF=True)
    if A is not None and B is not None:
        D = SpinOuterProduct(A, B, stack=True)
    
    if C is not None and D is not None:
        return np.append(D, C, axis=1)
    if C is not None:
        return C
    if D is not None:
        return D
    else:
        return None

print( ref4_sp )
print( SO_X_SF(ref4_sp, 2) )

[[1 1 0 0 0]
 [1 1 0 0 0]]
[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [1 1 1 1 0]
  [1 1 1 0 1]
  [1 1 0 1 1]]

 [[1 1 1 1 0]
  [1 1 1 0 1]
  [1 1 0 1 1]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]


In [ ]:
def SO_X(ref_SO, m, SF=False):

    if SF:
        C = None
        D = None
        ## +3 & -3
        A = MO_X(ref_SO[0],  m, SF=True)
        B = MO_X(ref_SO[1], -m, SF=True)
        if A is not None and B is not None:
            C = SpinOuterProduct(A, B, stack=True)
        
        ## -3 & +3
        A = MO_X(ref_SO[0], -m, SF=True)
        B = MO_X(ref_SO[1],  m, SF=True)
        if A is not None and B is not None:
            D = SpinOuterProduct(A, B, stack=True)
        
        if C is not None and D is not None:
            return np.append(D, C, axis=1)
        if C is not None:
            return C
        if D is not None:
            return D
        else:
            return None

    else:
        D     = None
        first = True
        for j in range(m+1):
            A = MO_X(ref_SO[0],   j)
            B = MO_X(ref_SO[1], m-j)
            if A is not None and B is not None:
                C = SpinOuterProduct(A, B, stack=True)
                if first:
                    D = C
                    first = False
                else:
                    D = np.append(D, C, axis=1)
            else:
                continue
        return D

SO_X(ref4_sp, 2, SF=False)

array([[[1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 1, 0],


# Reference Excitation

In [ ]:
CIm = [0,1,2]
SFm = [1,3]

def Ref_X(ref_SO, RAS=None, CIm=None, SFm=None):

    if RAS is None:
        RAS = np.arange(0, ref_SO.shape[1], 1, dtype=int)

    RASref = ref_SO[:, RAS]
    #loop   = Ext(reference, RASCI.shape[1]).swapaxes(1,2)
    

    ### CIm Loop
    if CIm is not None:
        first = True
        for i in range(len(CIm)):
            CIm_sIp = SO_X(RASref, CIm[i], SF=False)
            if first and CIm_sIp is not None:
                outCI = CIm_sIp
                first = False
                continue
            if CIm_sIp is not None:
                outCI = np.append(outCI, CIm_sIp, axis=1)
            else:
                continue
        out = Ext(ref_SO, outCI.shape[1])
        out[:,:,RAS]
        print(out.shape)

    ### SFm Loop
    if SFm is not None:
        first = True
        for i in range(len(SFm)):
            SFm_sIp = SO_X(RASref, SFm[i], SF=True)
            if first and CIm_sIp is not None:
                outSF = SFm_sIp
                first = False
                continue
            if SFm_sIp is not None:
                outSF = np.append(outSF, SFm_sIp, axis=1)
            else:
                continue
    


    return outCI, outSF

outCI, outSF = Ref_X(ref4_sp, CIm=CIm, SFm=SFm)
outSF.shape

(2, 5, 55)


(2, 12, 5)

# 

In [ ]:
ref4_sp

array([[1, 1, 0, 0, 0],
       [1, 1, 0, 0, 0]], dtype=int8)

!!! this is wrong!!!

In [ ]:
print(ref1_sp[0])
MO_Excitation(ref1_sp[0], 5, SF=False)

[1 1 1 1 1 0 0]


array([[0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [ ]:
len(np.where(ref1_sp[0] == 1)[0]) > j-1

True

Now we would like to feed the result from above into a CI class to compute nonzero elements of the CI Hamiltonian Matrix.

In [ ]:
class CIraw(object):
    # 1/10/22
    ''' A class to compute CI Matrix Elements '''
    def __init__(self, Binary):

        CI_dt = np.int16
        if len(Binary) < 125:
            CI_dt  = np.int8
        if len(Binary) > 32700:
            CI_dt  = np.int32

        Na   = np.einsum("Ip -> p", Binary[:,0,:])
        Nb   = np.einsum("Ip -> p", Binary[:,1,:])
        sign = np.cumsum( Binary, axis=2)
        for I in range(len(Binary)):
            ΛA = np.where(Binary[I,0,:]==1)[0]
            ΛB = np.where(Binary[I,1,:]==1)[0]
            sign[I, 0, ΛA] = np.arange(0, len(ΛA), 1)
            sign[I, 1, ΛB] = np.arange(0, len(ΛB), 1)

        self.sign = ( (-1)**(sign) ).astype(np.int8)

        ### basics
        Difference     = np.einsum("Isp, J -> IJsp", Binary, np.ones(len(Binary), dtype=np.int8)) - np.einsum("Isp, J -> JIsp", Binary, np.ones(len(Binary), dtype=np.int8))
        Sum            = np.einsum("Isp, J -> IJsp", Binary, np.ones(len(Binary), dtype=np.int8)) + np.einsum("Isp, J -> JIsp", Binary, np.ones(len(Binary), dtype=np.int8))
        SpinDifference = np.einsum("IJsp -> IJs", np.abs(Difference))//2

        print(SpinDifference.shape)

        I_A, J_A = np.where( np.all(SpinDifference==np.array([1,0], dtype=np.int8), axis=2) )
        I_B, J_B = np.where( np.all(SpinDifference==np.array([0,1], dtype=np.int8), axis=2) )
        self.I_A = I_A.astype(CI_dt)
        self.J_A = J_A.astype(CI_dt)
        self.I_B = I_B.astype(CI_dt)
        self.J_B = J_B.astype(CI_dt)


        CC_A = ( Binary[self.I_A, 0] + Binary[self.J_A, 0] )//2
        CC_B = ( Binary[self.I_B, 1] + Binary[self.J_B, 1] )//2
        print(CC_B.dtype)

        self.A_t  = (np.where( Difference[self.I_A, self.J_A, 0] ==  1 )[1]).astype(CI_dt)
        self.A    = (np.where( Difference[self.I_A, self.J_A, 0] == -1 )[1]).astype(CI_dt)
        self.B_t  = (np.where( Difference[self.I_B, self.J_B, 1] ==  1 )[1]).astype(CI_dt)
        self.B    = (np.where( Difference[self.I_B, self.J_B, 1] == -1 )[1]).astype(CI_dt)
        #self.CA_i = ((np.where( Sum[self.I_A, self.J_A, 0] == 2 )[1]).reshape(len(self.I_A), Na[0]-1)).astype(CI_dt)
        #self.CB_i = ((np.where( Sum[self.I_B, self.J_B, 1] == 2 )[1]).reshape(len(self.I_B), Nb[0]-1)).astype(CI_dt)

        



        I_AA, J_AA = np.where( np.all(SpinDifference==np.array([2,0], dtype=np.int8), axis=2) )
        I_BB, J_BB = np.where( np.all(SpinDifference==np.array([0,2], dtype=np.int8), axis=2) )
        I_AB, J_AB = np.where( np.all(SpinDifference==np.array([1,1], dtype=np.int8), axis=2) )
        self.I_AA = I_AA.astype(CI_dt)
        self.J_AA = J_AA.astype(CI_dt)
        self.I_BB = I_BB.astype(CI_dt)
        self.J_BB = J_BB.astype(CI_dt)
        self.I_AB = I_AB.astype(CI_dt)
        self.J_AB = J_AB.astype(CI_dt)

        self.AA   = (np.where( Difference[self.I_AA, self.J_AA, 0] == -1)[1].reshape(len(self.I_AA),2).T).astype(CI_dt)
        self.AA_t = (np.where( Difference[self.I_AA, self.J_AA, 0] ==  1)[1].reshape(len(self.I_AA),2).T).astype(CI_dt)
        self.BB   = (np.where( Difference[self.I_BB, self.J_BB, 1] == -1)[1].reshape(len(self.I_BB),2).T).astype(CI_dt)
        self.BB_t = (np.where( Difference[self.I_BB, self.J_BB, 1] ==  1)[1].reshape(len(self.I_BB),2).T).astype(CI_dt)
        self.AB   = np.asarray([ np.where( Difference[self.I_AB, self.J_AB, 0] == -1 )[1], np.where( Difference[self.I_AB, self.J_AB, 1] == -1 )[1] ], dtype=CI_dt)
        self.AB_t = np.asarray([ np.where( Difference[self.I_AB, self.J_AB, 0] ==  1 )[1], np.where( Difference[self.I_AB, self.J_AB, 1] ==  1 )[1] ], dtype=CI_dt)

    def H1CI(self, H1_so):

        Rule0  = np.einsum("Ip, pp -> I", Binary[:,0,:], H1_so[0])
        Rule0 += np.einsum("Ip, pp -> I", Binary[:,1,:], H1_so[1]) 
        H1CI   = np.diag(Rule0)
        if (self.I_A).size > 0:
            H1CI[self.I_A, self.J_A]  = H1_so[0, self.A, self.A_t]
            H1CI[self.I_A, self.J_A] *= self.sign[self.I_A, 0, self.A_t] * self.sign[self.J_A, 0, self.A]
        if (self.I_B).size > 0:
            H1CI[self.I_B, self.J_B]  = H1_so[1, self.B, self.B_t]
            H1CI[self.I_B, self.J_B] *= self.sign[self.I_B, 1, self.B_t] * self.sign[self.J_B, 1, self.B]


        return H1CI


    def H2CI(self, H2_so):

        Rule0 += np.einsum("Ip, Iq, ppqq -> I", Binary[:,0,:], Binary[:,0,:], H2_so[0,0])/2 
        Rule0 += np.einsum("Ip, Iq, ppqq -> I", Binary[:,1,:], Binary[:,0,:], H2_so[1,0])/2 
        Rule0 += np.einsum("Ip, Iq, ppqq -> I", Binary[:,0,:], Binary[:,1,:], H2_so[0,1])/2
        Rule0 += np.einsum("Ip, Iq, ppqq -> I", Binary[:,1,:], Binary[:,1,:], H2_so[1,1])/2 
        H2CI   = np.diag(Rule0)

        if (self.I_A).size > 0:
            H_CI[self.I_A, self.J_A] += np.einsum("In -> I", H2_so[0, 0, Ext(A, N_s[0]-1), Ext(A_t, Na[0]-1), self.CA_i, self.CA_i])
            H_CI[self.I_A, self.J_A] += np.einsum("Ii -> I", H2_so[0, 1, Ext(A, N_s[1]  ), Ext(A_t, Nb[0]  ), ΛB[I_A], ΛB[J_A]])
            H_CI[self.I_A, self.J_A] *= self.sign[self.I_A, 0, self.A_t] * self.sign[self.J_A, 0, self.A]

        if (self.I_B).size > 0:
            H_CI[self.I_B, self.J_B] += np.einsum("In -> I", H2_so[1, 1, Ext(self.B, N_s[1]-1), Ext(self.B_t, Nb[0]-1), self.CB_i, self.CB_i])
            H_CI[self.I_B, self.J_B] += np.einsum("Ii -> I", H2_so[1, 0, Ext(self.B, N_s[0]  ), Ext(self.B_t, Na[0]  ), self.ΛA[I_B], self.ΛA[J_B]])
            H_CI[self.I_B, self.J_B] *= self.sign[self.I_B, 1, self.B_t] * self.sign[self.J_B, 1, self.B]

        if (self.I_AA).size > 0:
            H2CI[self.I_AA, self.J_AA]  = H2_so[0, 0, self.AA[0], self.AA_t[0], self.AA[1], self.AA_t[1]]
            H2CI[self.I_AA, self.J_AA] *= self.sign[self.I_AA, 0, self.AA[0]] * self.sign[self.J_AA, 0, self.AA_t[0]] 
            H2CI[self.I_AA, self.J_AA] *= self.sign[self.I_AA, 0, self.AA[1]] * self.sign[self.J_AA, 0, self.AA_t[1]]

        if (self.I_BB).size > 0:
            H2CI[self.I_BB, self.J_BB]  = H2_so[1, 1, self.BB[0], self.BB_t[0], self.BB[1], self.BB_t[1]]
            H2CI[self.I_BB, self.J_BB] *= self.sign[self.I_BB, 1, self.BB[0]] * self.sign[self.J_BB, 1, self.BB_t[0]] 
            H2CI[self.I_BB, self.J_BB] *= self.sign[self.I_BB, 1, self.BB[1]] * self.sign[self.J_BB, 1, self.BB_t[1]]

        if (self.I_AB).size > 0:
            H2CI[self.I_AB, self.J_AB]  = H2_so[0, 1, self.AB[0], self.AB_t[0], self.AB[1], self.AB_t[1]]
            H2CI[self.I_AB, self.J_AB] *= self.sign[self.I_AB, 0, self.AB[0]] * self.sign[self.J_AB, 0, self.AB_t[0]] 
            H2CI[self.I_AB, self.J_AB] *= self.sign[self.I_AB, 1, self.AB[1]] * self.sign[self.J_AB, 1, self.AB_t[1]]


        return H2CI


    def HCI(self, H1_so, H2_so):
        return self.H1CI(H1_so) + self.H2CI(H2_so)

print( AA.shape )

asd = CIraw(AA.swapaxes(0,1))
asd.A_t

(2, 327, 7)
(327, 327, 2)
int8


ValueError: ignored

In [ ]:
Difference     = np.einsum("Isp, J -> IJsp", Binary, np.ones(len(Binary), dtype=np.int8)) - np.einsum("Isp, J -> JIsp", Binary, np.ones(len(Binary), dtype=np.int8))
Sum            = np.einsum("Isp, J -> IJsp", Binary, np.ones(len(Binary), dtype=np.int8)) + np.einsum("Isp, J -> JIsp", Binary, np.ones(len(Binary), dtype=np.int8))
SpinDifference = np.einsum("IJsp -> IJs", np.abs(Difference))//2

In [ ]:
( np.ones(5)/2 ).astype(np.int8)